In [ ]:
import matplotlib.pyplot as plt
from skimage.measure import find_contours, regionprops, label
from skimage import io
import cv2
import numpy as np
import skimage
import glob
from skimage import measure
from skimage.morphology import convex_hull_image
print(skimage.__version__)
import os
from PIL import Image
from IPython import display


In [ ]:
 def imsave(*args, **kwargs):
     args_list = list(args)
     for i in range(len(args_list)):
         if type(args_list[i]) != np.ndarray:
             print("Not a numpy array")
             return 0
         if len(args_list[i].shape) == 2:
             args_list[i] = np.dstack([args_list[i]]*3)
             if args_list[i].max() == 1:
                args_list[i] = args_list[i]*255

     out_destination = kwargs.get("out",'')
#      try:
     concatenated_arr = np.concatenate(args_list,axis=1)
     im = Image.fromarray(np.uint8(concatenated_arr))
#      except Exception as e:
#          print(e)
#          import ipdb; ipdb.set_trace()
#          return 0
#      if out_destination:
     print("Saving to %s"%(out_destination))
     im.save(out_destination)
#      else:
#         return im

def imshow(*args,**kwargs):
     """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
     Usage:
     imshow(img1, title="myPlot")
     imshow(img1,img2, title=['title1','title2'])
     imshow(img1,img2, cmap='hot')
     imshow(img1,img2,cmap=['gray','Blues']) """
     cmap = kwargs.get('cmap', 'gray')
     title= kwargs.get('title','')
     axis_off = kwargs.get('axis_off','')
     if len(args)==0:
         raise ValueError("No images given to imshow")
     elif len(args)==1:
         plt.title(title)
         plt.imshow(args[0], interpolation='none')
     else:
         n=len(args)
         if type(cmap)==str:
             cmap = [cmap]*n
         if type(title)==str:
             title= [title]*n
         plt.figure(figsize=(n*5,10))
         for i in range(n):
             plt.subplot(1,n,i+1)
             plt.title(title[i])
             plt.imshow(args[i], cmap[i])
             if axis_off:
               plt.axis('off')
     plt.show()
def TissueMaskGenerationPatch(patchRGB):
    '''
    Returns mask of tissue that obeys the threshold set by paip
    '''
    r = patchRGB[:,:,0] < 235
    g = patchRGB[:,:,1] < 210
    b = patchRGB[:,:,2] < 235
    tissue_mask = np.logical_or(r,np.logical_or(g,b))
    return tissue_mask 

In [ ]:
img_paths

In [ ]:
inference_id = 'test_final_final'
inference_dir = '../../results/saved_imgs/%s' % inference_id
img_paths = glob.glob(os.path.join(inference_dir,'ref_*.png'))
img_paths.sort()
list_fours = []
for i,img_path in enumerate(img_paths):
    img = cv2.imread(img_path)
    print(type(img))
    display.display(imsave(img))
    list_fours.append(int(input()))
    

In [ ]:
# Test set calc
inference_id = 'test_final_final'
inference_dir = '../../results/saved_imgs/%s' % inference_id
img_paths = glob.glob(os.path.join(inference_dir,'ref_*.png'))
img_paths.sort()
list_fours = [4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
print(list_fours)
prediction_csv_path = os.path.join(inference_dir,'prediction.csv')
# list_fours = []
def save_tumor_burden(img_path,tb):
    img_id = img_path.split('/')[-1].split('.')[0].split('_')[1]
    print(img_id)
    with open(prediction_csv_path,'a') as f:
        f.write('%s,%.2f\n'%(img_id,tb))
    
with open(prediction_csv_path,'w') as f:
    f.write('wsi_id,ratio\n')
    
for i,img_path in enumerate(img_paths):
    img = cv2.imread(img_path)
#     imshow(img,axis_off = True)
    x = list_fours[i]
    _,w,_ = img.shape
    if x == 5:
        w = w//x
        print(img.shape,w)
        viable_mask = np.average(img[:,w*3:w*4,:],axis=2)
        slide_img = img[:,4*w:,:]
    if x == 4:
        w = w//x
        print(img.shape,w)
        viable_mask = np.average(img[:,w*2:w*3,:],axis=2)
        slide_img = img[:,3*w:,:]
    
    np.place(viable_mask,viable_mask<128,0)
    np.place(viable_mask,viable_mask>=128,255)
    
    viable_mask_morph = viable_mask
    img_id = int(img_path.split('/')[-1].split('.')[0].split('_')[1])
    kernel = np.ones((20, 20), dtype=np.uint8)
    viable_mask_morph = cv2.morphologyEx(viable_mask, cv2.MORPH_CLOSE, kernel)
    if img_id<180 and img_id >170:
        print('biopsy', img_id)
        kernel = np.ones((5, 5), dtype=np.uint8)
    else:
        kernel = np.ones((15, 15), dtype=np.uint8)
    viable_mask_morph = cv2.morphologyEx(viable_mask_morph, cv2.MORPH_OPEN, kernel)
    viable_mask_morph = np.uint8(viable_mask_morph)

#     print(np.unique(viable_mask_morph))
#     r = regionprops(viable_mask_morph)
#     print(len(r))
#     xtl,ytl,xbr,ybr = r[0].bbox

#     wt = np.zeros_like(viable_mask)
#     wt[xtl:xbr,ytl:ybr] = 255
    convex_hull = np.uint8(convex_hull_image(viable_mask_morph))
    wt = np.uint8(convex_hull*TissueMaskGenerationPatch(slide_img))
    wt[wt>0] = 255

#     imsave(viable_mask_morph,viable_mask,wt,slide_img,axis_off=True)
    print("saving")
    out_path=os.path.join(inference_dir,'whole_tum_%s.jpeg'%img_path[-7:-4])
    print(imsave(wt,out=out_path))
#     print(np.unique(wt),np.unique(viable_mask)) 
#     _, contours, hierarchy = cv2.findContours(convex_hull,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
#     snake = active_contour(gaussian(rgb2gray(slide_img), 3),
#                            np.squeeze(contours[0]), alpha=0.15, beta=10, gamma=0.001,max_iterations=30)

#     fig = plt.figure(figsize=(7, 7))
#     ax = fig.add_subplot(111)
#     plt.gray()
#     ax.imshow(slide_img)
#     ax.plot(init[:, 0], init[:, 1], '--r', lw=3)
#     ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3)
#     ax.set_xticks([]), ax.set_yticks([])
#     ax.axis([0, slide_img.shape[1], slide_img.shape[0], 0])
#     plt.show()
    
    pred_whole_tum = np.sum(wt)
    viable_tum = np.sum(viable_mask)
    tumor_burden = viable_tum/pred_whole_tum*100
    print(tumor_burden)
    save_tumor_burden(img_path,tumor_burden)
    list_fours.append(x)

In [ ]:
# Training Set calc
inference_id = 'train_52'
inference_dir = '../../results/saved_imgs/%s' % inference_id
img_paths = glob.glob(os.path.join(inference_dir,'ref_*.png'))
img_paths.sort()
prediction_csv_path = os.path.join(inference_dir,'prediction.csv')
# list_fours = []
def save_tumor_burden(img_path,tb):
    img_id = img_path.split('/')[-1].split('.')[0].split('_')[1]
    print(img_id)
    with open(prediction_csv_path,'a') as f:
        f.write('%s,%.2f\n'%(img_id,tb))
    
with open(prediction_csv_path,'w') as f:
    f.write('wsi_id,ratio\n')
    
for i,img_path in enumerate(img_paths):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     imshow(img,axis_off = True)
    h,w,_ = img.shape
    w = w//5
    print(img.shape,w)
    viable_mask = np.average(img[:,w*2:w*3,:],axis=2)
    slide_img = img[:,4*w:,:]
    
    np.place(viable_mask,viable_mask<128,0)
    np.place(viable_mask,viable_mask>=128,255)
    
    viable_mask_morph = viable_mask
    kernel = np.ones((20, 20), dtype=np.uint8)
    viable_mask_morph = cv2.morphologyEx(viable_mask, cv2.MORPH_CLOSE, kernel)
    kernel = np.ones((15, 15), dtype=np.uint8)
    viable_mask_morph = cv2.morphologyEx(viable_mask_morph, cv2.MORPH_OPEN, kernel)
    viable_mask_morph = np.uint8(viable_mask_morph)

#     print(np.unique(viable_mask_morph))
#     r = regionprops(viable_mask_morph)
#     print(len(r))
#     xtl,ytl,xbr,ybr = r[0].bbox

#     wt = np.zeros_like(viable_mask)
#     wt[xtl:xbr,ytl:ybr] = 255
    convex_hull = np.uint8(convex_hull_image(viable_mask_morph))
    wt = np.uint8(convex_hull*TissueMaskGenerationPatch(slide_img))
    wt[wt>0] = 255

#     imsave(viable_mask_morph,viable_mask,wt,slide_img,axis_off=True)
    print("saving")
    sample_id = os.path.basename(img_path)[4:-4]
    out_path=os.path.join(inference_dir,'whole_tum_%s.jpeg'%sample_id)
    
    ref_img = np.transpose(cv2.imread('../../data/raw-data/ref_imgs/%s_ref.png'%sample_id),[1,0,2])
    gt = np.average(ref_img[h*2:h*3,:,:],axis=2)
    print(img.shape)
    print(ref_img.shape)
    print(gt.shape)
    print(wt.shape)
    print(viable_mask.shape)
    print(imsave(viable_mask,wt,gt,out=out_path))
#     print(np.unique(wt),np.unique(viable_mask)) 
#     _, contours, hierarchy = cv2.findContours(convex_hull,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
#     snake = active_contour(gaussian(rgb2gray(slide_img), 3),
#                            np.squeeze(contours[0]), alpha=0.15, beta=10, gamma=0.001,max_iterations=30)

#     fig = plt.figure(figsize=(7, 7))
#     ax = fig.add_subplot(111)
#     plt.gray()
#     ax.imshow(slide_img)
#     ax.plot(init[:, 0], init[:, 1], '--r', lw=3)
#     ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3)
#     ax.set_xticks([]), ax.set_yticks([])
#     ax.axis([0, slide_img.shape[1], slide_img.shape[0], 0])
#     plt.show()
    
    pred_whole_tum = np.sum(wt)
    viable_tum = np.sum(viable_mask)
    tumor_burden = viable_tum/pred_whole_tum*100
    print(tumor_burden)
    save_tumor_burden(img_path,tumor_burden)

In [ ]:
os.path.join(inference_dir,'whole_tum_%d.png'%i)

In [ ]:
np.squeeze(contours[0]).shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour

# Test scipy version, since active contour is only possible
# with recent scipy version
import scipy
split_version = scipy.__version__.split('.')
if not(split_version[-1].isdigit()): # Remove dev string if present
        split_version.pop()
scipy_version = list(map(int, split_version))
new_scipy = scipy_version[0] > 0 or \
            (scipy_version[0] == 0 and scipy_version[1] >= 14)

img = data.astronaut()
img = rgb2gray(img)

s = np.linspace(0, 2*np.pi, 400)
x = 220 + 100*np.cos(s)
y = 100 + 100*np.sin(s)
init = np.array([x, y]).T

if not new_scipy:
    print('You are using an old version of scipy. '
          'Active contours is implemented for scipy versions '
          '0.14.0 and above.')

if new_scipy:
    snake = active_contour(gaussian(img, 3),
                           init, alpha=0.015, beta=10, gamma=0.001)

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)
    plt.gray()
    ax.imshow(img)
    ax.plot(init[:, 0], init[:, 1], '--r', lw=3)
    ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3)
    ax.set_xticks([]), ax.set_yticks([])
    ax.axis([0, img.shape[1], img.shape[0], 0])

In [ ]:
len(list_fours)

In [ ]:
b = label(img)
print(np.unique(b))
plt.imshow(b)

In [ ]:
np.unique(img_raw)

In [ ]:
actual_whole_tum = np.sum(whole_tum)/255 
pred_whole_tum = np.sum(wt)/255
viable_tum = np.sum(img_raw)/255
print(actual_whole_tum,pred_whole_tum,viable_tum)
print(viable_tum/actual_whole_tum,viable_tum/pred_whole_tum)

In [ ]:
for prop in r:
    for a in prop:
        print(a,prop[a])

In [ ]:
# Find contours at a constant value of 0.8
contours = measure.find_contours(img, 0.8)

# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)

for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
aa,contours,_ = cv2.findContours(img.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
out = np.zeros_like(img)
cv2.drawContours(out,contours,-1,255,3)
imshow(out,img,aa)

In [ ]:
print(np.unique(out))
Image.fromarray(out.astype('uint8'))